In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd



In [2]:
#load data
data = pd.read_csv('Symptom2Disease.csv')

#split data into train and test
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

print(len(train_data))
print(f"{len(test_data)}, {test_data.shape})")

for i,x in enumerate(test_data):
    if(i == 10):
        break
    print(x)
    


960
240, (240, 3))
Unnamed: 0
label
text


In [3]:
#connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
#train data
X_train = train_data['text'].values
y_train = train_data['label'].values

#test data
X_test = test_data['text'].values
y_test = test_data['label'].values

In [5]:
#tokenize data and pad them to equal sequences

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

#turning it into a set suppositly makes it faster
stopwords_set = set(stopwords.words())


#Preprocess function will remove stopwords, punctuation, lowercase the text
#Might add stemming and lemmatization later on
def preprocess(text):

    #lowercase
    text = word_tokenize(text)

    #remove stopwords and punctuation
    processed_text = [word.lower() for word in text if not word.lower() in stopwords_set and word.isalpha()]
    

    return processed_text

print(preprocess("I am going to SCHOOL. Where is my bag, lunch, and books?"))
print(f"\nBefore: {X_train[0]}")

#preprcoess all data
X_train = [preprocess(text) for text in X_train]
X_test = [preprocess(text) for text in X_test]

print(f"Processed: {X_train[0]}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bobth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bobth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['school', 'bag', 'lunch', 'books']

Before: I have seen rashes on my arms and neck and it itches if I scratch them. I've also had a high fever for a few days. I have no idea what is causing it. The itching is causing me a lot of discomforts.
Processed: ['rashes', 'arms', 'neck', 'itches', 'scratch', 'high', 'fever', 'days', 'idea', 'causing', 'itching', 'causing', 'lot', 'discomforts']


In [6]:
#imports
from nltk.probability import FreqDist

#Basically encoding the text

#vocab creation
word_frequency = FreqDist([word.lower() for text in X_train for word in text])

# Create the vocabulary by assigning a unique index to each word
vocab = {word: idx+1 for idx, (word, _) in enumerate(word_frequency.items())}

# encode text
def encode(text, vocab):
    encoded = []
    for word in text:
        encoded.append(vocab.get(word,0))
    return encoded

#encode all data
print(X_train[0])
print(X_train[4])
X_train = [encode(text, vocab) for text in X_train]
X_test = [encode(text, vocab) for text in X_test]

print(f"Encoded: {X_train[0]}")
print(f"Encoded:{X_train[4]}")


['rashes', 'arms', 'neck', 'itches', 'scratch', 'high', 'fever', 'days', 'idea', 'causing', 'itching', 'causing', 'lot', 'discomforts']
['breathing', 'issues', 'persistent', 'cough', 'exhaustion', 'coughing', 'lot', 'thick', 'mucoid', 'sputum', 'high', 'fever', 'feeling', 'exhausted', 'tired', 'cope']
Encoded: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 10, 12, 13]
Encoded:[52, 53, 54, 55, 56, 57, 12, 58, 59, 60, 6, 7, 61, 62, 63, 64]


In [7]:
#pad sequences
from torch.nn.utils.rnn import pad_sequence

#pad sequences
X_train = pad_sequence([torch.tensor(text) for text in X_train], batch_first=True)
X_test = pad_sequence([torch.tensor(text) for text in X_test], batch_first=True)


In [8]:
#encoding labels

label_mapping = {label: i for i, label in enumerate(np.unique(y_train))}
y_train = np.array([label_mapping[label] for label in y_train])
y_test = np.array([label_mapping[label] for label in y_test])


In [20]:
#convert to tensors
print(X_train.dtype)
print(y_train.dtype)

X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)
 
print(f"training text: {X_train.shape}")
print(f"training labels: {y_train.shape}")

print(f"testing text: {X_test.shape}")
print(f"testing labels: {y_test.shape}")

print(X_train[0])



torch.int64
torch.int32
training text: torch.Size([960, 25])
training labels: torch.Size([960])
testing text: torch.Size([240, 24])
testing labels: torch.Size([240])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 10, 12, 13,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])


C:\Users\bobth\AppData\Local\Temp\ipykernel_23480\2028079522.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\bobth\AppData\Local\Temp\ipykernel_23480\2028079522.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train)
C:\Users\bobth\AppData\Local\Temp\ipykernel_23480\2028079522.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\bobth\AppData\Local\Temp\ipykernel_23480\2028079522.py:9: UserWarning: To copy construct from a tensor

In [10]:
#create a simple fnn model that takes in the input size
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out
    
#rnn model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    
#LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    def predict(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    
    




#### 24 represents the 24 unique diseases in the data set and 25 represents to length of each encoded vector


In [22]:
#define hyperparameters
learning_rate = 0.001
epochs = 10

input_size = X_train.shape[1]
hidden_size = 20
output_size = len(label_mapping)

print(f"Input size: {input_size}")
print(f"out size: {output_size}")




Input size: 25
out size: 24


In [23]:
model_num = 1
model = None
if(model_num == 0):
    model = FNN(input_size, hidden_size, output_size)

elif(model_num == 1):
    #Reshape for RNN
    model = RNN(input_size, hidden_size, output_size)


model.to(device)

RNN(
  (rnn): RNN(25, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=24, bias=True)
  (softmax): Softmax(dim=0)
)

In [24]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
model.train()
#train the model
for epoch in range(epochs):
    for i in range(len(X_train)):
        #get data
        inputs = X_train[i].unsqueeze(0).float().to(device)
        labels = y_train[i].unsqueeze(0).long().to(device)
        
        #forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i % 100 == 0):
            print(f"Epoch: {epoch} Loss: {loss.item()}")
    

        
    


IndexError: too many indices for tensor of dimension 2

In [ ]:
model.eval()
#test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(0, seq_len_test):
        inputs = padded_test_input_ids.to(device)
        labels = padded_test_labels.to(device)
        
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, actual = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == actual).sum().item()
        
    print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')



Accuracy of the network on the 240 test images: 100.0 %


In [ ]:
#f1score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

with torch.no_grad():
    correct = 0
    total = 0
    y_pred = []
    y_true = []
    for i in range(0, seq_len_test):
        inputs = padded_test_input_ids.to(device)
        labels = padded_test_labels.to(device)
        
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, actual = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == actual).sum().item()
        y_pred.append(predicted)
        y_true.append(actual)
    
    #[y_pred, y_true] = [torch.cat(y_pred, dim=0).cpu().numpy(), torch.cat(y_true, dim=0).cpu().numpy()]

    print(y_pred)
    print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')
    print(f'f1 score: {f1_score(y_true, y_pred, average="macro")}')
    print(f'precision score: {precision_score(y_true, y_pred, average="macro")}')
    print(f'recall score: {recall_score(y_true, y_pred, average="macro")}')

[tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([0

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

: 